# Custom Evaluation Criteria

In this example, we will see how to create a custom metric.

We will create a metric that evaluates whether a user query is decomposed into sub-queries, covering all the angles of the original query to retrieve all the necessary information.

# Creating the metric

`flow-judge` makes it easy to create custom metrics.

In [1]:
from flow_judge.metrics import CustomMetric, RubricItem
from IPython.display import Markdown, display

# Define the criteria
evaluation_criteria = """Do the generated sub-queries provide sufficient breadth to cover all aspects of the main query?"""

# Define the rubric using RubricItem's
rubric = [
    RubricItem(
        score=1,
        description="The sub-queries lack breadth and fail to address multiple important aspects of the main query. They are either too narrow, focusing on only one or two dimensions of the question, or they diverge significantly from the main query's intent. Using these sub-queries alone would result in a severely limited exploration of the topic."),
    RubricItem(
        score=2,
        description="The sub-queries cover some aspects of the main query but lack comprehensive breadth. While they touch on several dimensions of the question, there are still noticeable gaps in coverage. Some important facets of the main query are either underrepresented or missing entirely. Answering these sub-queries would provide a partial, but not complete, exploration of the main topic."),
    RubricItem(
        score=3,
        description="The sub-queries demonstrate excellent breadth, effectively covering all major aspects of the main query. They break down the main question into a diverse set of dimensions, ensuring a comprehensive exploration of the topic. Each significant facet of the main query is represented in the sub-queries, allowing for a thorough and well-rounded investigation of the subject matter."),
]

# We need to define the required inputs and output for the metric
required_inputs = ["query"]
required_output = "sub_queries"

# Create the metric
sub_query_coverage = CustomMetric(
    name="sub-query-coverage",
    criteria=evaluation_criteria,
    rubric=rubric,
    required_inputs=required_inputs,
    required_output=required_output
)

As you can see, we quickly created a custom metric that will instruct the model to evaluate according to the criteria and rubric we set.

In [2]:
sub_query_coverage

CustomMetric(name='sub-query-coverage', criteria='Do the generated sub-queries provide sufficient breadth to cover all aspects of the main query?', rubric=[RubricItem(score=1, description="The sub-queries lack breadth and fail to address multiple important aspects of the main query. They are either too narrow, focusing on only one or two dimensions of the question, or they diverge significantly from the main query's intent. Using these sub-queries alone would result in a severely limited exploration of the topic."), RubricItem(score=2, description='The sub-queries cover some aspects of the main query but lack comprehensive breadth. While they touch on several dimensions of the question, there are still noticeable gaps in coverage. Some important facets of the main query are either underrepresented or missing entirely. Answering these sub-queries would provide a partial, but not complete, exploration of the main topic.'), RubricItem(score=3, description='The sub-queries demonstrate exce

# Running evaluations with the custom metric

Now, we just need to initialize the judge with our custom metric and run the evaluations.

In [3]:
from flow_judge.models.model_factory import ModelFactory
from flow_judge.flow_judge import EvalInput, FlowJudge

# Create a model using ModelFactory
model = ModelFactory.create_model("Flow-Judge-v0.1-AWQ")

# Initialize the judge
judge = FlowJudge(metric=sub_query_coverage, model=model)

INFO 09-20 15:53:15 awq_marlin.py:89] The model is convertible to awq_marlin during runtime. Using awq_marlin kernel.
WARNING 09-20 15:53:15 config.py:378] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 09-20 15:53:15 llm_engine.py:213] Initializing an LLM engine (v0.6.0) with config: model='flowaicom/Flow-Judge-v0.1-AWQ', speculative_config=None, tokenizer='flowaicom/Flow-Judge-v0.1-AWQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq_marlin, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_back

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 09-20 15:53:17 model_runner.py:926] Loading model weights took 2.1861 GB
INFO 09-20 15:53:19 gpu_executor.py:122] # GPU blocks: 3080, # CPU blocks: 682


In [4]:
# Sample to evaluate

query = "I placed an order for a custom-built gaming PC (order #AC-789012) two weeks ago, but the estimated delivery date has changed twice since then. Originally it was supposed to arrive yesterday, then it got pushed to next week, and now the tracking page shows 'Status: Processing' with no delivery estimate at all. I've tried calling customer service, but after waiting on hold for an hour, I was told to check the website. Can you please look into this and explain what's causing the delays, when I can realistically expect my order to arrive, and whether I'm eligible for any kind of compensation or expedited shipping given these repeated delays? I'm especially concerned because I need this computer for an upcoming gaming tournament I'm participating in next month."
sub_queries = "What is the current shipping status of my order? When will my computer arrive?"

# bad decomposition
eval_input = EvalInput(
    inputs=[
        {"query": query}
    ],
    output={"sub_queries": sub_queries}
)

# Run the evaluation
result = judge.evaluate(eval_input)

Processed prompts: 100%|██████████| 1/1 [00:07<00:00,  7.44s/it, est. speed input: 132.19 toks/s, output: 47.87 toks/s]


In [6]:
# Display the result
display(Markdown(f"__Feedback:__\n{result.feedback}\n\n__Score:__\n{result.score}"))

__Feedback:__
The generated sub-queries demonstrate excellent breadth and effectively cover all major aspects of the main query. The first sub-query, "What is the current shipping status of my order?" directly addresses the customer's concern about the changing delivery dates and the current 'Status: Processing' message on the tracking page. This sub-query ensures that the customer's immediate concern about the order's location is addressed.

The second sub-query, "When will my computer arrive?" tackles the customer's request for a realistic delivery estimate, which is crucial given the repeated delays and the customer's urgent need for the gaming PC. This sub-query aims to provide the specific information the customer is seeking regarding the expected arrival time of their order.

Furthermore, the sub-queries implicitly cover the customer's concern about potential compensation or expedited shipping by addressing the current status and expected delivery time. While not explicitly mentioned, these aspects are inherently part of understanding the order's status and delivery timeline.

The sub-queries also touch on the customer's need for the computer for an upcoming gaming tournament, as addressing the delivery status and providing a realistic arrival time would be directly relevant to the customer's situation.

Overall, the sub-queries break down the main question into a diverse set of dimensions, ensuring a comprehensive exploration of the topic. Each significant facet of the main query is represented, allowing for a thorough and well-rounded investigation of the subject matter.

__Score:__
3

In [7]:
# Good decomposition

sub_queries = """1. What is the current status of order #AC-789012, and why has it changed from the original estimated delivery date?
2. What specific factors are causing the delays in processing and shipping this custom-built gaming PC?
3. Based on the current situation, when can the customer realistically expect the order to arrive?
4. Given the repeated delays and changes in estimated delivery, what compensation options (if any) are available to the customer?
5. Is expedited shipping an option at this point, and if so, how would it affect the delivery timeline?
6. How can the urgency of this order be communicated and prioritized, considering the customer's upcoming gaming tournament next month?
7. What steps has customer service already taken to address this issue, and what additional actions can be taken to resolve it?
8. How can the customer receive more frequent and accurate updates about their order status going forward?"""

eval_input = EvalInput(
    inputs=[
        {"query": query}
    ],
    output={"sub_queries": sub_queries}
)

# Run the evaluation
result = judge.evaluate(eval_input)

Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.74s/it, est. speed input: 175.59 toks/s, output: 46.46 toks/s]


In [8]:
# Display the result
display(Markdown(f"__Feedback:__\n{result.feedback}\n\n__Score:__\n{result.score}"))

__Feedback:__
The generated sub-queries demonstrate excellent breadth, effectively covering all major aspects of the main query. Each significant facet of the main query is represented in the sub-queries, ensuring a comprehensive exploration of the topic.

The sub-queries address the following key points from the main query:

1. Current status and reasons for delivery date changes
2. Specific factors causing delays
3. Realistic expected delivery time
4. Compensation options due to delays
5. Availability of expedited shipping
6. Prioritization of the order considering the upcoming tournament
7. Actions taken by customer service and potential additional steps
8. Improved communication and updates for the customer

These sub-queries break down the main question into diverse dimensions, ensuring a thorough investigation of the subject matter. They cover the technical aspects of the order, the customer service response, potential solutions, and future improvements. This comprehensive approach allows for a well-rounded exploration of the topic, addressing all major concerns raised in the original query.

The sub-queries are well-structured and cover all important aspects without straying from the main query's intent. They provide a solid foundation for investigating and resolving the customer's issues, ensuring that no significant part of the original query is overlooked.

__Score:__
3

In [9]:
# Okish decomposition

sub_queries = """1. What is the current status of order #AC-7892?
Why has the delivery date changed multiple times?
When will the custom-built gaming PC actually arrive?
What are your store hours on weekends?
How can I get better customer service support?
8. Can the order be expedited?
9. What is the name of the person that is responsible for the company?
"""

eval_input = EvalInput(
    inputs=[
        {"query": query}
    ],
    output={"sub_queries": sub_queries}
)

# Run the evaluation
result = judge.evaluate(eval_input)
# Display the result
display(Markdown(f"__Feedback:__\n{result.feedback}\n\n__Score:__\n{result.score}"))

Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.70s/it, est. speed input: 158.41 toks/s, output: 45.84 toks/s]


__Feedback:__
The generated sub-queries demonstrate a good attempt to cover various aspects of the main query, but they fall short of providing comprehensive breadth. The sub-queries address several key points raised in the main query, such as the current status of the order, reasons for delivery date changes, expected arrival time, and the possibility of expediting the order. These are directly relevant to the customer's concerns.

However, there are noticeable gaps in coverage. The sub-queries fail to address the customer's repeated attempts to contact customer service and the specific instructions given by customer service. They also don't address the customer's concern about eligibility for compensation due to repeated delays. While the sub-queries do touch on the need for better customer service support, they don't specifically address the customer's frustration with the support experience.

Additionally, the sub-queries include some that are not directly relevant to the main query, such as questions about store hours and the name of the person responsible for the company. These do not contribute to addressing the customer's specific concerns about the order and its delays.

Overall, while the sub-queries cover some important aspects of the main query, they miss several key points and include some irrelevant questions, preventing a truly comprehensive exploration of the topic.

__Score:__
2